#### 1. Import modules

In [1]:
import sys, os
import numpy as np
import pandas as pd
from pathlib import PurePath
# For importing the deployment modules
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"parent dir: {parent_dir}")
sys.path.append(parent_dir)

parent dir: /home/bras2024/ic/journal_repo/N_Deployment_journal/heterogeneous_v2i_sequential_deployment_by_grasp


#### 2. Trace dataframe

In [2]:
trace_path = PurePath(".") / ".." / ".." / "datasets" / "6_to_8am.csv"

trace_df = pd.read_csv(trace_path, names = ['vehicle_id','time_instant','x','y','cell_time', 'null'], sep=";")
trace_df.drop(columns=["null"], inplace=True)

#### 3. Cells dataframe

In [3]:
import numpy as np

# Para facilitar na identificação das células
trace_df["x:y"] = trace_df["x"].astype("str") + ":"+  trace_df["y"].astype("str")

cells_df = pd.DataFrame(data={
    "Inicio_c": trace_df[["vehicle_id", "x:y"]].groupby("vehicle_id").first()["x:y"].value_counts(),
    "Fim_c": trace_df[["vehicle_id", "x:y"]].groupby("vehicle_id").last()["x:y"].value_counts(),
    "Tempo_veic": trace_df[["cell_time", "x:y"]].groupby("x:y").mean()["cell_time"],
    "Pop": trace_df["x:y"].value_counts(),
}).fillna(0)
# Fillna porque algumas células não estão no inicio ou fim das trajetórias

cells_df["Inicio_c"] = cells_df["Inicio_c"].astype(np.int64)
cells_df["Fim_c"] = cells_df["Fim_c"].astype(np.int64)

trace_df = trace_df.drop("x:y", axis=1)

#### 4. Clustering

loading precomputed clusters

In [4]:
import pickle
with open(PurePath(".") / ".." / ".." / "clustering_results" / 'clustering_results.pkl', 'rb') as f:
    clustering_res = pickle.load(f)
print(clustering_res)

[+] silhouette score:
		0.6847897547538015
[+] clusters sizes:
[+] sizes:
		2: 402
		1: 1001
		0: 2822
[+] summary:
		          Inicio_c                     Tempo_veic                      \
              mean        std min  max       mean       std       min   
cluster                                                                 
0         3.784904   5.606386   0   19   1.776783  1.162007  1.000000   
1        34.370629  10.082038  20   55   2.275417  0.940324  1.000000   
2        75.696517  18.968209  56  171   2.718574  1.217375  1.118922   

                    
               max  
cluster             
0        20.569476  
1         7.925000  
2        12.569100  


#### 5. Sequential deployment via GRASP v2

* Clusters are traversed in the order 2, 1, 0 (refering to the labels). They are the central, intermediate and margin clusters;
* Scenarios have fixed sequential N requirements of 9, 3 and 1;

    * Each scenario is identified by a sequence of TAU requirements, the same as the genetic algorithm input;
    * For each scenario, a total infrastructure size ($\rho$) is specified and divided for each cluster. If deployment in one cluster results in less RSUs, by the smaller cluster size or no solutions finds, the method tries to deploy the remaining number of RSUs in the next clusters;

* For each scenario, a folder is created, in which executions corresponding to different total rhos are saved. Similarly to v1, the overall infrastructure is saved;

**homogenous_v2i_grasp_based_solution/sequential/n-deployment must've been compiled**

**number of iterations for grasp is set as only 1**

In [5]:
from pathlib import Path

def run_total_rho_and_tau_seq(trace_df: pd.DataFrame, cells_df: pd.DataFrame, total_rho: int, clusters_TAU: tuple[int]):

    n_deployment_path = PurePath(".") / ".." / ".." / "homogenous_v2i_grasp_based_solution" / "sequential" / "n-deployment"
    trace_path = PurePath(".") / ".." / ".." / "datasets" / "6_to_8am.csv"

    clusters_deployments_p = []

    clusters_iterator = [2, 1, 0]

    clusters_rsus = [(total_rho // len(clusters_iterator)) for _ in clusters_iterator]

    clusters_N = [9, 3, 1]

    grasp_n_iter = 1

    from common_modules import n_deployment_functions
    for i, cluster in enumerate(clusters_iterator):

        clusters_deployments_p.append(
            n_deployment_functions.n_deployment_param(
                executable_path=n_deployment_path,
                num_rsus=clusters_rsus[i],
                tau=clusters_TAU[i],
                rcl_len=15,
                n_iter=grasp_n_iter,
                num_cont=clusters_N[i],
                seed=1 + cluster,
                trace_path=trace_path)
        )

    from common_modules import infra_eval
    progressive_cov_metrics = []
    for i, cluster in enumerate(clusters_iterator):
        progressive_cov_metrics.append(
            infra_eval.n_contacts_metric(N=clusters_N[i], tau=clusters_TAU[i])
        )

    from common_modules import cells_progressive_clustering_deployment_v2

    clusters_deployment_p = cells_progressive_clustering_deployment_v2.clusters_deployment_param(
        clusters_iterator=clusters_iterator,
        deployments_param=clusters_deployments_p,
        progressive_coverage_metrics=progressive_cov_metrics
    )

    # OVERALL INFRA EVALUATION PARAMETERS ==============
    # ==================================================

    overall_metrics = []
    for i, cluster in enumerate(clusters_iterator):
        overall_metrics.append(
            infra_eval.n_contacts_metric(N=clusters_N[i], tau=clusters_TAU[i]))
    overall_infra_evaluation_p = infra_eval.overall_infra_evaluation_param(metrics=overall_metrics)

    # EXECUTE ==========================================
    # ==================================================

    try:

        tau_scenario_folder = Path("scenario_tau=" + ",".join([str(tau) for tau in clusters_TAU]))

        rho_execution_folder = tau_scenario_folder / (f"execution_rho={total_rho}")
        import shutil
        try:
            shutil.rmtree(rho_execution_folder)
        except FileNotFoundError:
            pass

        os.makedirs(rho_execution_folder, exist_ok=True)

        p_c_d_res = cells_progressive_clustering_deployment_v2.run_progressive_cells_clustering_deployment(
            trace_df=trace_df,
            cells_features_df=cells_df,

            clustering_precomp_res=clustering_res,
            clusters_deployment_p=clusters_deployment_p,
            overall_infra_evaluation_p=overall_infra_evaluation_p,

            log=False,
            delete_generated_files=True,
            clusters_generated_files_directory=None
        )

        from functools import reduce
        overall_infra = reduce(lambda x, y: set(x) | set(y), p_c_d_res.results["clusters_deployment"].clusters_deployment_infras.values(), [])

        with open(rho_execution_folder / "overall_infra.csv", mode="w") as f:
            for cell in overall_infra:
                f.write(f"{cell[0]},{cell[1]}\n")

    except Exception as e:
        print(e)

In [6]:
run_total_rho_and_tau_seq(trace_df=trace_df, cells_df=cells_df, total_rho=600, clusters_TAU=(30, 30, 30))
run_total_rho_and_tau_seq(trace_df=trace_df, cells_df=cells_df, total_rho=600, clusters_TAU=(120, 120, 120))